![Assimila logo](../../helpers/logosmall.png "Assimila")
# Assimila Peatlands Demonstrator
# Compare bounding box for 2 time steps

If the drop down menus do not appear restart the notebook by selecting "Kernel" then "Restart" above.

In [1]:
import os
import sys
sys.path.append("../../helpers")

from data import Data
from maptools import MapTools
from widgets import Widgets

import ipywidgets as widgets
import matplotlib
%matplotlib notebook
# matplotlib.use("nbagg")
from IPython.display import clear_output

from ipyleaflet import (Map, Marker, basemaps, basemap_to_tiles, 
                        TileLayer, ImageOverlay, Polyline, Polygon, 
                        Rectangle,  GeoJSON, WidgetControl,  DrawControl, 
                        LayerGroup, FullScreenControl, interactive)

import ipywidgets as widgets
import datetime
from IPython.core.display import HTML

display(HTML("""
<style>
.container { 
    width:60% !important; 
}
</style>
"""))

display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
</style>'''))

def on_button_clicked(b):
    button.disabled = True
    d = Data(out)
    m1 = d.color_map_nesw_compare_reduced(product.value, 
                                  subproduct.value, 
                                  north.value,
                                  east.value, 
                                  south.value, 
                                  west.value,
                                  date1.value, 
                                  date2.value)
    button.disabled = False
        
def update_north(x):
    """"
    Updates the 'North' coordinate display by linking widgets. 
    
    :param x: update value
    """    
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (north, 'value'))

def update_east(x):
    """"
    Updates the 'East' coordinate display by linking widgets. 
    
    :param x: update value
    """    
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (east, 'value'))

def update_south(x):
    """"
    Updates the 'South' coordinate display by linking widgets. 
    
    :param x: update value
    """
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (south, 'value'))

def update_west(x):
    """"
    Updates the 'West' coordinate display by linking widgets. 
    
    :param x: update value
    """ 
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (west, 'value'))    
    

def handle_draw(self, action, geo_json):
    """
    Handles drawing of layers on the map by extracting coordinates
    and updating the relevant coordinate display box.
    
    :param action: 
    :param geo_json: 
    
    :return:
    """
    w = Widgets()
    d = Data(out)
    
    s = geo_json.get('geometry','type')
    t = s.get('type')
    
    if t == 'Point':
        north, east, south, west = MapTools.get_coords_point(geo_json)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)

    if t == 'Polygon':
        north, east, south, west = MapTools.get_coords_polygon(geo_json)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)

def update(*args):
    subproduct.options = w.get_subproduct_list(product.value)
    
m = MapTools(center=(55, 0), zoom=4, width='400px', height='400px')
m.map.add_control(m.dc)
m.prepare_map()      

w = Widgets()
product = w.product()
subproduct = w.subproduct()
projection = w.projection()

north = w.get_point(60.0, 'North (lat): ')
east = w.get_point(2.5, 'East (lon): ')
south = w.get_point((49), 'South (lat): ')
west = w.get_point((-13), 'West (lon): ')
date1 = w.get_date(datetime.date(2018,1,1), 'Date 1: ')
date2 = w.get_date(datetime.date(2018,1,17), 'Date 2: ')
button = w.set_up_button(on_button_clicked, "Get data")
w.display_widget_bounding_box(product, subproduct, north, east, south, west, 
                              date1, date2, button, m.map)

product.layout = widgets.Layout(width='auto', height='auto')
subproduct.layout = widgets.Layout(width='auto', height='auto')
north.layout = widgets.Layout(width='auto', height='auto')
east.layout = widgets.Layout(width='auto', height='auto')
south.layout = widgets.Layout(width='auto', height='auto')
west.layout = widgets.Layout(width='auto', height='auto')
date1.layout = widgets.Layout(width='auto', height='auto')
date2.layout = widgets.Layout(width='auto', height='auto')

label = widgets.Label()
m.mouse_interaction(label)
product.observe(update)

out = w.display_output()

m.dc.on_draw(handle_draw)

update()
product.observe(update)

out = w.display_output()

Label(value='')

Output()

Output()

## Available MODIS products:

  - __MOD11A1 - Land surface temperature__
  - __MOD13A2 - Vegetation indices__
  - __MCD43A3 - Albedo__


## MOD11A1 - Land surface temperature
Daily, 1km resolution
  - Day and night __land surface temperature__ (LST_Day_1km, LST_night_1km)

Also:
 - __Quality control flags__ (day and night).
   - These need interperating before using.
 - Day and night __view time__.
 - Day and night __view angle__.
 - __Clear sky coverage__ (day/night).
 - __Emissivity__ (band 31 and 32)


## MOD13A2 - Vegetation indices
Every 16 days, 1 km resolution
  - Vegetation indices: __NDVI__, __EVI__

Also:
  - Vegetation index __quality indicators__
  - __Reflectance__ for red, NIR, blue and MIR bands
  - Viewing and sun angles: __view zenith__, __sun zenith__ and __relative azimuth__
  - __Pixel reliability__

## MCD43A3 - Albedo
Daily, 500 m resolution
  - __White sky albedo__ for each band
  - __Black sky albedo__ for each band
  - __Quality indicators__
    - need interpretation